# Лабораторная работа 4

In [1]:
#%tensorflow_version 2.x
import tensorflow as tf
device = tf.test.gpu_device_name()
if device != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device))

Found GPU at: /device:GPU:0


In [2]:
USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

Using device:  /device:GPU:0


Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [3]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы. 

In [4]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [5]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [6]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети. 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [7]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации. 

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 2)
2. Функция активации ReLU 
3. Сверточный слой (3 x 3 kernels, zero-padding = 1)
4. Функция активации ReLU 
5. Полносвязный слой 
6. Функция активации Softmax 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [8]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        self.conv1 = tf.keras.layers.Conv2D(channel_1, 5,
                                            padding='same',
                                            activation='relu',
                                            )
        self.conv2 = tf.keras.layers.Conv2D(channel_2, 3,
                                            padding='same',
                                            activation='relu',
                                            )
        self.fc = tf.keras.layers.Dense(num_classes,
                                        activation='softmax',
                                        )
        self.flatten = tf.keras.layers.Flatten()
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        scores = self.fc(x)


        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return scores

In [9]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [10]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    with tf.device(device):

        
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [11]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.048341751098633, Accuracy: 6.25, Val Loss: 3.0606014728546143, Val Accuracy: 13.199999809265137
Iteration 100, Epoch 1, Loss: 2.2526936531066895, Accuracy: 28.078588485717773, Val Loss: 1.8996381759643555, Val Accuracy: 37.400001525878906
Iteration 200, Epoch 1, Loss: 2.079308271408081, Accuracy: 32.462684631347656, Val Loss: 1.7915847301483154, Val Accuracy: 38.900001525878906
Iteration 300, Epoch 1, Loss: 2.0019450187683105, Accuracy: 34.31789779663086, Val Loss: 1.854075312614441, Val Accuracy: 37.0
Iteration 400, Epoch 1, Loss: 1.9309961795806885, Accuracy: 36.17908477783203, Val Loss: 1.73353111743927, Val Accuracy: 41.0
Iteration 500, Epoch 1, Loss: 1.8855661153793335, Accuracy: 37.181888580322266, Val Loss: 1.6666195392608643, Val Accuracy: 42.89999771118164
Iteration 600, Epoch 1, Loss: 1.8562592267990112, Accuracy: 37.986167907714844, Val Loss: 1.6913576126098633, Val Accuracy: 41.60000228881836
Iteration 700, Epoch 1, Loss: 1.8311114311218262, Ac

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 . 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 0.5 .

In [12]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(
        learning_rate=learning_rate,
        momentum=0.9,
        nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.283267021179199, Accuracy: 18.75, Val Loss: 2.342148542404175, Val Accuracy: 10.800000190734863
Iteration 100, Epoch 1, Loss: 1.879780650138855, Accuracy: 33.24566650390625, Val Loss: 1.651153564453125, Val Accuracy: 42.20000076293945
Iteration 200, Epoch 1, Loss: 1.7345596551895142, Accuracy: 38.611629486083984, Val Loss: 1.4760578870773315, Val Accuracy: 49.599998474121094
Iteration 300, Epoch 1, Loss: 1.6480772495269775, Accuracy: 41.51785659790039, Val Loss: 1.424744725227356, Val Accuracy: 50.80000305175781
Iteration 400, Epoch 1, Loss: 1.5794930458068848, Accuracy: 43.92534255981445, Val Loss: 1.3599034547805786, Val Accuracy: 51.70000076293945
Iteration 500, Epoch 1, Loss: 1.5330846309661865, Accuracy: 45.59006881713867, Val Loss: 1.3055338859558105, Val Accuracy: 56.400001525878906
Iteration 600, Epoch 1, Loss: 1.5032511949539185, Accuracy: 46.66181564331055, Val Loss: 1.2973215579986572, Val Accuracy: 54.599998474121094
Iteration 700, Epoch 1, Los

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [13]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.0299243927001953, Accuracy: 10.9375, Val Loss: 2.994493246078491, Val Accuracy: 11.0
Iteration 100, Epoch 1, Loss: 2.2488906383514404, Accuracy: 29.11509895324707, Val Loss: 1.8926331996917725, Val Accuracy: 39.099998474121094
Iteration 200, Epoch 1, Loss: 2.078331232070923, Accuracy: 32.975746154785156, Val Loss: 1.8675116300582886, Val Accuracy: 39.29999923706055
Iteration 300, Epoch 1, Loss: 1.9978171586990356, Accuracy: 34.61378860473633, Val Loss: 1.88007652759552, Val Accuracy: 37.79999923706055
Iteration 400, Epoch 1, Loss: 1.9293904304504395, Accuracy: 36.40507888793945, Val Loss: 1.7179672718048096, Val Accuracy: 42.5
Iteration 500, Epoch 1, Loss: 1.8842988014221191, Accuracy: 37.40019989013672, Val Loss: 1.6630563735961914, Val Accuracy: 43.79999923706055
Iteration 600, Epoch 1, Loss: 1.8532198667526245, Accuracy: 38.290348052978516, Val Loss: 1.698543906211853, Val Accuracy: 42.5
Iteration 700, Epoch 1, Loss: 1.8274450302124023, Accuracy: 38.959

Альтернативный менее гибкий способ обучения:

In [14]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.6559 - sparse_categorical_accuracy: 0.4315


[1.6559083461761475, 0.43149998784065247]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [15]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    input_shape = (32, 32, 3)
    channel_1, channel_2, num_classes = 32, 16, 10
    # захотелось разнообразия, добавила инициалайзер
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Conv2D(channel_1, 5, padding='same',
                               activation='relu', kernel_initializer=initializer,
                               input_shape=input_shape),
        tf.keras.layers.Conv2D(channel_2, 3, padding='same',
                               activation='relu', kernel_initializer=initializer),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes,
                              activation='softmax', kernel_initializer=initializer)
    ]
    model = tf.keras.Sequential(layers)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(
        learning_rate=learning_rate,
        momentum=0.9,
        nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.816711187362671, Accuracy: 15.625, Val Loss: 2.5846240520477295, Val Accuracy: 13.799999237060547
Iteration 100, Epoch 1, Loss: 1.9769022464752197, Accuracy: 30.213489532470703, Val Loss: 1.762237310409546, Val Accuracy: 38.70000076293945
Iteration 200, Epoch 1, Loss: 1.8434154987335205, Accuracy: 35.16013717651367, Val Loss: 1.6460498571395874, Val Accuracy: 43.39999771118164
Iteration 300, Epoch 1, Loss: 1.774012565612793, Accuracy: 37.448089599609375, Val Loss: 1.5977330207824707, Val Accuracy: 44.20000076293945
Iteration 400, Epoch 1, Loss: 1.7166632413864136, Accuracy: 39.46384048461914, Val Loss: 1.5500742197036743, Val Accuracy: 45.400001525878906
Iteration 500, Epoch 1, Loss: 1.6783416271209717, Accuracy: 40.82147979736328, Val Loss: 1.4912668466567993, Val Accuracy: 47.5
Iteration 600, Epoch 1, Loss: 1.652477741241455, Accuracy: 41.77152633666992, Val Loss: 1.468161702156067, Val Accuracy: 48.5
Iteration 700, Epoch 1, Loss: 1.629252552986145, Accu

In [16]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.2999 - sparse_categorical_accuracy: 0.5388


[1.299912929534912, 0.5388000011444092]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры. 

Ниже представлен пример для полносвязной сети. 

In [17]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


In [18]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.9012584686279297, Accuracy: 10.9375, Val Loss: 2.803870677947998, Val Accuracy: 11.300000190734863
Iteration 100, Epoch 1, Loss: 2.2308576107025146, Accuracy: 28.62004852294922, Val Loss: 1.918965458869934, Val Accuracy: 36.70000076293945
Iteration 200, Epoch 1, Loss: 2.0755867958068848, Accuracy: 32.06623077392578, Val Loss: 1.8362841606140137, Val Accuracy: 37.900001525878906
Iteration 300, Epoch 1, Loss: 1.9990183115005493, Accuracy: 33.82474899291992, Val Loss: 1.864819884300232, Val Accuracy: 37.79999923706055
Iteration 400, Epoch 1, Loss: 1.9299622774124146, Accuracy: 35.653053283691406, Val Loss: 1.7135390043258667, Val Accuracy: 41.70000076293945
Iteration 500, Epoch 1, Loss: 1.8866677284240723, Accuracy: 36.71718978881836, Val Loss: 1.6585328578948975, Val Accuracy: 41.70000076293945
Iteration 600, Epoch 1, Loss: 1.8564320802688599, Accuracy: 37.68198776245117, Val Loss: 1.6913734674453735, Val Accuracy: 40.099998474121094
Iteration 700, Epoch 1, 

- **Filter size**: Above we used 5x5 and 3x3; is this optimal?
- **Number of filters**: Above we used 16 and 32 filters. Would more or fewer do better?
- **Pooling**: We didn't use any pooling above. Would this improve the model?
- **Normalization**: Would your model be improved with batch normalization, layer normalization, group normalization, or some other normalization strategy?
- **Network architecture**: The ConvNet above has only three layers of trainable parameters. Would a deeper model do better?
- **Global average pooling**: Instead of flattening after the final convolutional layer, would global average pooling do better? This strategy is used for example in Google's Inception network and in Residual Networks.
- **Regularization**: Would some kind of regularization improve performance? Maybe weight decay or dropout?

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут). 

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [20]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        self.conv1_1 = tf.keras.layers.Conv2D(16, (5, 5), padding="same", activation='relu')
        self.conv1_2 = tf.keras.layers.Conv2D(16, (5, 5), padding="same", activation='relu')
        self.norm1 = tf.keras.layers.BatchNormalization(axis=1)
        self.pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))

        self.conv2_1 = tf.keras.layers.Conv2D(32, (3, 3), padding="same", activation='relu')
        self.conv2_2 = tf.keras.layers.Conv2D(32, (3, 3), padding="same", activation='relu')
        self.norm2 = tf.keras.layers.BatchNormalization(axis=1)
        self.pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))

        self.flatten = tf.keras.layers.Flatten()
        self.dense3 = tf.keras.layers.Dense(512, activation='relu')
        self.norm3 = tf.keras.layers.BatchNormalization()
        self.drop = tf.keras.layers.Dropout(0.5)

        self.dense4 = tf.keras.layers.Dense(10, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1_1(input_tensor)
        x = self.conv1_2(x)
        x = self.norm1(x)
        x = self.pool1(x)

        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.norm2(x)
        x = self.pool2(x)

        x = self.flatten(x)
        x = self.dense3(x)
        x = self.norm3(x)
        x = self.drop(x)

        x = self.dense4(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 3.6868598461151123, Accuracy: 7.8125, Val Loss: 2.288907527923584, Val Accuracy: 13.799999237060547
Iteration 700, Epoch 1, Loss: 1.6026955842971802, Accuracy: 46.05251693725586, Val Loss: 1.268660306930542, Val Accuracy: 55.0
Iteration 1400, Epoch 2, Loss: 1.0632737874984741, Accuracy: 62.70423126220703, Val Loss: 1.1362545490264893, Val Accuracy: 59.20000076293945
Iteration 2100, Epoch 3, Loss: 0.9266433715820312, Accuracy: 67.73396301269531, Val Loss: 1.1207503080368042, Val Accuracy: 61.599998474121094
Iteration 2800, Epoch 4, Loss: 0.8367145657539368, Accuracy: 70.74739074707031, Val Loss: 1.109132170677185, Val Accuracy: 62.30000305175781
Iteration 3500, Epoch 5, Loss: 0.7689974308013916, Accuracy: 73.3659896850586, Val Loss: 0.8558132648468018, Val Accuracy: 69.80000305175781
Iteration 4200, Epoch 6, Loss: 0.6971839666366577, Accuracy: 75.49696350097656, Val Loss: 0.8719804286956787, Val Accuracy: 70.0
Iteration 4900, Epoch 7, Loss: 0.6489784717559814

Опишите все эксперименты, результаты. Сделайте выводы.

Первая самая ленивая идея - использовать трёхслойную сеть, которая была реализована раньше. На третьей эпохе были достигнуты многообещающие 62% точности на валидационной выборке, но... не вышло.

Вторая проба - после свёрточных слоёв вставить нормализацию по батчам и сделать макспулинг. Очень плохо, даже до 40% не дотянуло. :с

Третья попытка - три слоя с нормализацией и макспулингом. Фильтры 16, 32, 64. К десятой эпохе точность на валидации 12%, на обучающей - почти 80%. Грустно.

Четыре. Добавила kernel_regularizer. Лучше не стало.

Пять. Убрала регуляризацию. Добавила ещё один полносвязный слой перед выходным (512 нейрончиков). Очень многообещающее начало, добрались до 47% к шестой эпохе, на десятой вернулись к 21%.

ШЕСТЬ. Возникла идея добавить ещё свёрточных слоёв таким образом, чтобы нормализация и пулинг происходили через каждые два слоя. После первого полносвязного слоя добавила dropout. 3, 2, 1... Удача! 

Сами свёрточные слои немного поправила - у первых двух 16 фильтров, у второй пары - по 32.

Комментарий: в первые несколько попыток я совершила довольно нехорошую ошибку, когда пыталась переиспользовать слои, так что полученные результаты нельзя назвать релевантными. 

В целом до использования дропаута иногда появлялось что-то, похожее на переобучение - точность на трейне была очень хорошей, а точность на валидации падала до значений меньше 10%. 

В целом добавление новых слоёв, пулинг, нормализация и дропауты положительно сказываются на качестве моделей, но нельзя забывать про баланс точности/производительности (чем больше слоёв, тем большее время требуется для расчётов).